## Build Dataframe from XML

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
content = []
with open("SecondHAREM.xml", "r", encoding='UTF-8') as file:
    content = file.readlines()
    content = "".join(content)
    soup = bs(content, "lxml")

In [3]:
def process_text(text):
    text = " ".join(text.split())
    text = text.replace("\n", "")
    text = text.strip()
    return text

In [4]:
data = []

index_p = 0

for index_doc, doc in enumerate(soup.find_all('doc')):
    _id_doc = None
    
    if "docid" in doc.attrs:
        _id_doc = doc.attrs["docid"]
                                
    for p in doc.find_all('p'):
        sentence_text_raw = p.text
        sentence_text_processed = process_text(sentence_text_raw)
        entities = []
        
        for index_em, em in enumerate(p.find_all('em')):
            _entity = process_text(em.text)
            _id = None
            _categ = None
            _tipo = None
            _tiporel = None
            _corel = None
            _subtipo = None
            if "id" in em.attrs:
                _id = em.attrs["id"]
            if "categ" in em.attrs:
                _categ = em.attrs["categ"]
            if "tipo" in em.attrs:
                _tipo = em.attrs["tipo"]
            if "subtipo" in em.attrs:
                _subtipo = em.attrs["subtipo"]
            if "tiporel" in em.attrs:
                _tiporel = em.attrs["tiporel"]
            if "corel" in em.attrs:
                _corel = em.attrs["corel"]
            
            # Remove entities that do not have a category 
            if (_categ):
                d = {
                    "doc_id": _id_doc,
                    "doc_index": index_doc,
                    "p_sentence_processed": sentence_text_processed,
                    "p_index": index_p,
                    "entity": _entity,
                    "entity_id": _id,
                    "entity_index": index_em,
                    "categ": _categ,
                    "tipo": _tipo,
                    "subtipo": _subtipo,
                    "tipo_final": None,
                    "tiporel": _tiporel,
                    "corel": _corel
                }
                data.append(d)
        index_p += 1
        
df = pd.DataFrame(data)


## Process Dataframe

#### 1- Remove all relations involving entities with the ALT tag

In [5]:
for row, value in enumerate(df['corel'].tolist()):
    # if corel is not null
    if (value):
        # separate each corel object into a list
        corel_list = value.split()
        tiporel_list = df.at[row, 'tiporel'].split()
        
        final_corel_list = []
        final_tiporel_list = []

        for corel_index, corel in enumerate(corel_list):
            # if the corel object ends with "a", "b", "c" or "d" then it is an ALT entity
            # remove it from the 'corel' and 'tiporel' list
            if not (corel.endswith(("a", "b", "c", "d"))):
                final_corel_list.append(corel)
                if (corel_index < len(tiporel_list)):
                    final_tiporel_list.append(tiporel_list[corel_index])
            
        df.at[row, 'corel'] = " ".join(final_corel_list)
        df.at[row, 'tiporel'] = " ".join(final_tiporel_list)

#### 2- Format the relations that have additional data (ex: ACONTECIMENTO\*\*outrarel\*\*H2-dftre765-102\*\*OUTRO will just be "outrarel")

In [6]:
for row, value in enumerate(df['tiporel'].tolist()):
    # if tiporel is not null
    if (value):
        # separate each relation into a list
        tiporel_list = value.split()
        
        final_tiporel_list = []

        for tiporel_index, tiporel in enumerate(tiporel_list):
            if ("*" in tiporel):
                words_list = tiporel.split("**")
                relation = words_list[1]
                final_tiporel_list.append(relation)
            else:
                final_tiporel_list.append(tiporel)
            
            df.at[row, 'tiporel'] = " ".join(final_tiporel_list)

#### 3- Simplify the Category, Type and Subtype into a final class

VALOR = valor

TEMPO = tempo

PESSOA + INDIVIDUAL or CARGO = individuo

PESSOA + GRUPOIND or GRUPOCARGO or GRUPOMEMBRO or POVO or MEMBRO = grupo

OUTRO = outro

ORGANIZACAO = organizacao

OBRA = obra

LOCAL + VIRTUAL + OBRA = obra

LOCAL + VIRTUAL = virtual

LOCAL + resto = local

COISA = coisa

ACONTECIMENTO = acontecimento

ABSTRACCAO + DISCIPLINA = disciplina

ABSTRACCAO + resto = abstraccao

In [7]:
def get_final_type(categ, tipo, subtipo):
    if (categ == "VALOR"):
        return "valor"
    elif (categ == "TEMPO"):
        return "tempo"
    elif (categ == "OUTRO"):
        return "outro"
    elif (categ == "ORGANIZACAO"):
        return "organizacao"
    elif (categ == "OBRA"):
        return "obra"
    elif (categ == "COISA"):
        return "coisa"
    elif (categ == "ACONTECIMENTO"):
        return "acontecimento"
    elif (categ == "PESSOA"):
        if (tipo == "INDIVIDUAL" or tipo == "CARGO"):
            return "individuo"
        else:
            return "grupo"
    elif (categ == "LOCAL"):
        if (tipo == "VIRTUAL"):
            if (subtipo == "OBRA"):
                return "obra"
            else:
                return "virtual"
        else:
            return "local"
    elif (categ == "ABSTRACCAO"):
        if (tipo == "DISCIPLINA"):
            return "disciplina"
        else:
            return "abstraccao"

In [8]:
for row, value in enumerate(df['categ'].tolist()):
    # separate each category into a list (each entity can have more than one category)
    categ_list = value.split("|")
    
    # get the types and subtypes related to the category (check if there is a type and subtype)
    tipo_list = []
    if (df.at[row, 'tipo']):
        tipo_list = df.at[row, 'tipo'].split("|")
    subtipo_list = []
    if (df.at[row, 'subtipo']):
        subtipo_list = df.at[row, 'subtipo'].split("|")

    final_type_list = []

    # for each category in the list, take the type and subtype (if any) and infer the final class
    for categ_index, categ in enumerate(categ_list):
        tipo = None
        if (categ_index < len(tipo_list)):
            tipo = tipo_list[categ_index]
            
        subtipo = None
        if (categ_index < len(subtipo_list)):
            subtipo = subtipo_list[categ_index]
        
        final_type = get_final_type(categ, tipo, subtipo)
        
        final_type_list.append(final_type)

    df.at[row, 'tipo_final'] = "|".join(final_type_list)

#### 4- Remove the multi-types, each entity will have ONE and ONLY ONE final class, in the following priority:
    1º: individuo
    2º: virtual
    3º: local
    4º: organizacao
    5º: grupo
    6º: obra
    7°: acontecimento
    8°: disciplina
    9°: tempo
    10°: valor
    11°: abstraccao
    12°: coisa
    13°: outro

In [9]:
priority_list = [
    "individuo",
    "virtual",
    "local",
    "organizacao",
    "grupo",
    "obra",
    "acontecimento",
    "disciplina",
    "tempo",
    "valor",
    "abstraccao",
    "coisa",
    "outro",
]

for row, value in enumerate(df['tipo_final'].tolist()):
    # separate each final class into a list (each entity can have more than one final class)
    tipo_list = value.split("|")
    
    result_index = 99
    
    for tipo in tipo_list:
        current_priority_index = priority_list.index(tipo)
        
        # update the final class only if the new one has more priority
        if (current_priority_index < result_index):
            result_index = current_priority_index
    
    result = priority_list[result_index]

    df.at[row, 'tipo_final'] = result

## Show entity class distribution

#### DISTRIBUTION WITH 13 CLASSES

In [10]:
individuo_cnt = 0
virtual_cnt = 0
local_cnt = 0
organizacao_cnt = 0
grupo_cnt = 0
obra_cnt = 0
acontecimento_cnt = 0
disciplina_cnt = 0
tempo_cnt = 0
valor_cnt = 0
abstraccao_cnt = 0
coisa_cnt = 0
outro_cnt = 0
total_cnt = 0

for tipo in df['tipo_final']:
    total_cnt += 1
    
    if (tipo == "individuo"):
        individuo_cnt += 1
    elif (tipo == "virtual"):
        virtual_cnt += 1
    elif (tipo == "local"):
        local_cnt += 1
    elif (tipo == "organizacao"):
        organizacao_cnt += 1
    elif (tipo == "grupo"):
        grupo_cnt += 1
    elif (tipo == "obra"):
        obra_cnt += 1
    elif (tipo == "acontecimento"):
        acontecimento_cnt += 1
    elif (tipo == "disciplina"):
        disciplina_cnt += 1
    elif (tipo == "tempo"):
        tempo_cnt += 1
    elif (tipo == "valor"):
        valor_cnt += 1
    elif (tipo == "abstraccao"):
        abstraccao_cnt += 1
    elif (tipo == "coisa"):
        coisa_cnt += 1
    elif (tipo == "outro"):
        outro_cnt += 1
        
print("TOTAL = " + str(total_cnt))
print("INDIVIDUO = " + str(individuo_cnt))
print("VIRTUAL = " + str(virtual_cnt))
print("LOCAL = " + str(local_cnt))
print("ORGANIZACAO = " + str(organizacao_cnt))
print("GRUPO = " + str(grupo_cnt))
print("OBRA = " + str(obra_cnt))
print("ACONTECIMENTO = " + str(acontecimento_cnt))
print("DISCIPLINA = " + str(disciplina_cnt))
print("TEMPO = " + str(tempo_cnt))
print("VALOR = " + str(valor_cnt))
print("ABSTRACCAO = " + str(abstraccao_cnt))
print("COISA = " + str(coisa_cnt))
print("OUTRO = " + str(outro_cnt))

TOTAL = 7817
INDIVIDUO = 1774
VIRTUAL = 59
LOCAL = 1480
ORGANIZACAO = 1106
GRUPO = 353
OBRA = 489
ACONTECIMENTO = 323
DISCIPLINA = 182
TEMPO = 1199
VALOR = 352
ABSTRACCAO = 110
COISA = 311
OUTRO = 79


#### REDUCE THE NUMBER OF CLASSES: NEW DISTRIBUTION WITH 9 CLASSES

In [11]:
individuo_cnt = 0
local_cnt = 0
organizacao_cnt = 0
obra_cnt = 0
acontecimento_cnt = 0
tempo_cnt = 0
valor_cnt = 0
abstraccao_cnt = 0
outro_cnt = 0
total_cnt = 0

for row, tipo in enumerate(df['tipo_final'].tolist()):
    total_cnt += 1
    
    if (tipo == "individuo"):
        individuo_cnt += 1
    elif (tipo == "virtual"):
        df.at[row, 'tipo_final'] = "local"
        local_cnt += 1
    elif (tipo == "local"):
        local_cnt += 1
    elif (tipo == "organizacao"):
        organizacao_cnt += 1
    elif (tipo == "grupo"):
        df.at[row, 'tipo_final'] = "organizacao"
        organizacao_cnt += 1
    elif (tipo == "obra"):
        obra_cnt += 1
    elif (tipo == "acontecimento"):
        acontecimento_cnt += 1
    elif (tipo == "disciplina"):
        df.at[row, 'tipo_final'] = "abstraccao"
        abstraccao_cnt += 1
    elif (tipo == "tempo"):
        tempo_cnt += 1
    elif (tipo == "valor"):
        valor_cnt += 1
    elif (tipo == "abstraccao"):
        abstraccao_cnt += 1
    elif (tipo == "coisa"):
        df.at[row, 'tipo_final'] = "outro"
        outro_cnt += 1
    elif (tipo == "outro"):
        outro_cnt += 1
        
print("TOTAL = " + str(total_cnt))
print("INDIVIDUO = " + str(individuo_cnt))
print("LOCAL = " + str(local_cnt))
print("ORGANIZACAO = " + str(organizacao_cnt))
print("OBRA = " + str(obra_cnt))
print("ACONTECIMENTO = " + str(acontecimento_cnt))
print("TEMPO = " + str(tempo_cnt))
print("VALOR = " + str(valor_cnt))
print("ABSTRACCAO = " + str(abstraccao_cnt))
print("OUTRO = " + str(outro_cnt))

TOTAL = 7817
INDIVIDUO = 1774
LOCAL = 1539
ORGANIZACAO = 1459
OBRA = 489
ACONTECIMENTO = 323
TEMPO = 1199
VALOR = 352
ABSTRACCAO = 292
OUTRO = 390


## Process relations types (remove or replace some relations)

In [12]:
entity_ids = df['entity_id']

for entity_id in entity_ids:
    # EX: tiporels_for_each_entity == autor_de natural_de participante_em
    tiporels_for_each_entity = df.loc[df.entity_id == entity_id, "tiporel"].values[0]
    
    if (tiporels_for_each_entity):
        # EX: tiporel_list = [autor_de, natural_de, participante_em]
        tiporel_list = tiporels_for_each_entity.split()
        
        corel_list = df.loc[df.entity_id == entity_id, "corel"].values[0].split()
        
        final_tiporel_list = []
        final_corel_list = []
        
        for index, tiporel in enumerate(tiporel_list):
            
            # rename the relation
            if (tiporel == "local_nascimento_de"):
                final_tiporel_list.append("natural_de")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de"):
                final_tiporel_list.append("ident")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_ident"):
                final_tiporel_list.append("ident")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_obra_de"):
                final_tiporel_list.append("obra_de")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_data_de"):
                final_tiporel_list.append("datado_de")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_incluido"):
                final_tiporel_list.append("incluido")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_inclui"):
                final_tiporel_list.append("inclui")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_pratica_se"):
                final_tiporel_list.append("pratica_se")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_praticado_por"):
                final_tiporel_list.append("praticado_por")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_outrarel"):
                final_tiporel_list.append("outrarel")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "nome_de_vinculo_inst"):
                final_tiporel_list.append("vinculo_inst")
                final_corel_list.append(corel_list[index])
            elif (tiporel == "ocorre_em"):
                final_tiporel_list.append("localizado_em")
                final_corel_list.append(corel_list[index])
                
            # completely remove the relation
            elif (tiporel == "consequencia_de"
                  or tiporel == "localizacao_de"
                  or tiporel == "local_morte"
                  or tiporel == "nomeado_por"
                  or tiporel == "outra_edicao"
                  or tiporel == "periodo_vida"
                  or tiporel == "residente_de"):
                print(tiporel)
                
            # keep the relation
            else:
                final_tiporel_list.append(tiporel)
                final_corel_list.append(corel_list[index])
                
        df.loc[df.entity_id == entity_id, "corel"] = " ".join(final_corel_list)
        df.loc[df.entity_id == entity_id, "tiporel"] = " ".join(final_tiporel_list)

df.to_csv("dataset_processed.csv")  

residente_de
residente_de
outra_edicao
localizacao_de
localizacao_de
nomeado_por
nomeado_por
nomeado_por
nomeado_por
periodo_vida
local_morte
consequencia_de
outra_edicao
periodo_vida
outra_edicao
residente_de
local_morte
periodo_vida
periodo_vida
nomeado_por
local_morte
periodo_vida
local_morte
localizacao_de
localizacao_de
localizacao_de


## Print entities and relations

#### Entities distribution

In [17]:
dic = {}
for categ in df["tipo_final"]:
    if(categ == categ):
        categlist = categ.split("|")
        for entity in categlist:
            if(entity in dic):
                dic[entity] += 1
            else:
                dic[entity] = 1
                
keys = list(dic.keys())
keys.sort()
for tipo in keys:
    print("TIPO={}\tTOTAL={}".format(tipo, dic[tipo]))

TIPO=abstraccao	TOTAL=292
TIPO=acontecimento	TOTAL=323
TIPO=individuo	TOTAL=1774
TIPO=local	TOTAL=1539
TIPO=obra	TOTAL=489
TIPO=organizacao	TOTAL=1459
TIPO=outro	TOTAL=390
TIPO=tempo	TOTAL=1199
TIPO=valor	TOTAL=352


#### Relations distribution

In [24]:
dic = {}

for tiporel in df["tiporel"]:
    if(tiporel):
        tipolist = tiporel.split()
        for tipo in tipolist:
            if ("*" in tipo):
                words_list = tiporel.split("**")
                relation = words_list[1]
            else:
                relation = tipo
            counter +=1
            if(relation in dic):
                dic[relation] += 1
            else:
                dic[relation] = 1
                
keys = list(dic.keys())
keys.sort()
for tipo in keys:
    print("TIPO={}\tTOTAL={}".format(tipo, dic[tipo]))

TIPO=autor_de	TOTAL=54
TIPO=causador_de	TOTAL=22
TIPO=data_de	TOTAL=76
TIPO=data_morte	TOTAL=9
TIPO=data_nascimento	TOTAL=6
TIPO=datado_de	TOTAL=10
TIPO=ident	TOTAL=2201
TIPO=inclui	TOTAL=320
TIPO=incluido	TOTAL=507
TIPO=localizado_em	TOTAL=121
TIPO=natural_de	TOTAL=133
TIPO=obra_de	TOTAL=93
TIPO=outrarel	TOTAL=97
TIPO=participante_em	TOTAL=90
TIPO=personagem_de	TOTAL=14
TIPO=pratica_se	TOTAL=16
TIPO=praticado_em	TOTAL=42
TIPO=praticado_por	TOTAL=16
TIPO=praticante_de	TOTAL=26
TIPO=produtor_de	TOTAL=28
TIPO=produzido_por	TOTAL=22
TIPO=propriedade_de	TOTAL=18
TIPO=proprietario_de	TOTAL=20
TIPO=relacao_familiar	TOTAL=82
TIPO=relacao_profissional	TOTAL=17
TIPO=residencia_de	TOTAL=15
TIPO=sede_de	TOTAL=196
TIPO=ter_participacao_de	TOTAL=64
TIPO=vinculo_inst	TOTAL=256
